In [ ]:
import os

import torch
from tqdm import tqdm
from transformers import (
    RobertaForSequenceClassification,
)

In [ ]:
# (k_value, best_steps)
best_ckpts = [
    (0, 500),
    (1, 550),
    (2, 375),
    (3, 325),
    (4, 350),
    (5, 475),
    (6, 500),
    (7, 525),
    (8, 525),
    (9, 400),
]

In [ ]:
"""
convert huggingface ckpt to fairseq ckpt
"""

temp_dir = "temp_ckpts/k10"
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)
    
for idx, ckpt_idx in tqdm(best_ckpts):
    path = "outputs/roberta-large_k10/{}/checkpoint-{}".format(idx, ckpt_idx)
    model = RobertaForSequenceClassification.from_pretrained(path)
    torch.save(model.state_dict(), os.path.join(temp_dir, "checkpoint{}.pt".format(idx)))

In [ ]:
"""
average checkpoints
"""
!python average_checkpoints.py \
    --inputs=temp_ckpts/k10 \
    --output=temp_ckpts/k10/checkpoint_average.pt \
    --num-epoch-checkpoints=10

In [ ]:
"""
convert fairseq ckpt to huggingface ckpt
"""
output_dir = os.path.join("outputs/roberta-large_k10", "checkpoint-average")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
model = RobertaForSequenceClassification.from_pretrained("roberta-large", num_labels=6)
model.load_state_dict(torch.load(os.path.join(temp_dir, "checkpoint_average.pt"))["model"])
model.save_pretrained(output_dir)

In [ ]:
os.rmdir(temp_dir)